<a href="https://colab.research.google.com/github/eestevekpmp/credit-risk-classification/blob/main/credit_risk_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%env HV_DOC_HTML=True

env: HV_DOC_HTML=True


In [2]:
pip install -q hvplot

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 26.5 MB/s eta 0:00:00


In [3]:
pip install -q holoviews

In [4]:
# Import the modules
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report

---

## Split the Data into Training and Testing Sets

### Step 1: Read the `lending_data.csv` data from the `Resources` folder into a Pandas DataFrame.

In [12]:
# Read the CSV file from the Resources folder into a Pandas DataFrame
lending_data_DF = pd.read_csv(
    "lending_data.csv"
)

# Review the DataFrame
lending_data_DF.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,52800,0.431818,5,1,22800,0
1,8400.0,6.692,43600,0.311927,3,0,13600,0
2,9000.0,6.963,46100,0.349241,3,0,16100,0
3,10700.0,7.664,52700,0.430740,5,1,22700,0
4,10800.0,7.698,53000,0.433962,5,1,23000,0


### Step 2: Create the labels set (`y`)  from the “loan_status” column, and then create the features (`X`) DataFrame from the remaining columns.

In [13]:
# Separate the data into labels and features

# Separate the y variable, the labels
y = lending_data_DF["loan_status"]

# Separate the X variable, the features
x = lending_data_DF.drop(columns = "loan_status")

In [15]:
# Review the y variable Series
y.head()

0    0
1    0
2    0
3    0
4    0
Name: loan_status, dtype: int64

In [16]:
# Review the X variable DataFrame
x.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
0,10700.0,7.672,52800,0.431818,5,1,22800
1,8400.0,6.692,43600,0.311927,3,0,13600
2,9000.0,6.963,46100,0.349241,3,0,16100
3,10700.0,7.664,52700,0.430740,5,1,22700
4,10800.0,7.698,53000,0.433962,5,1,23000


### Step 3: Check the balance of the labels variable (`y`) by using the `value_counts` function.

In [17]:
# Check the balance of our target values
y.value_counts()

0    75036
1     2500
Name: loan_status, dtype: int64

### Step 4: Split the data into training and testing datasets by using `train_test_split`.

In [18]:
# Import the train_test_learn module
from sklearn.model_selection import train_test_split

# Split the data using train_test_split
# Assign a random_state of 1 to the function
x_train, x_test, y_train, y_test = train_test_split(x,y,random_state=1,stratify=y)

---

## Create a Logistic Regression Model with the Original Data

###  Step 1: Fit a logistic regression model by using the training data (`X_train` and `y_train`).

In [20]:
# Import the LogisticRegression module from SKLearn
from sklearn.linear_model import LogisticRegression

# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
classifierModel = LogisticRegression(solver='lbfgs', random_state=1)
classifierModel
# Fit the model using training data
classifierModel.fit(x_train,y_train)

LogisticRegression(random_state=1)

### Step 2: Save the predictions on the testing data labels by using the testing feature data (`X_test`) and the fitted model.

In [21]:
# Make a prediction using the testing data
classifierModel_predictions = classifierModel.predict(x_test)

### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [28]:
# Print the balanced_accuracy score of the model 
balanced_accuracy_score(y_test,classifierModel_predictions)


0.9442676901753825

In [29]:
# Generate a confusion matrix for the model
confusion_matrix(y_test,classifierModel_predictions)

array([[18679,    80],
       [   67,   558]])

In [31]:
# Print the classification report for the model
print(classification_report(y_test,classifierModel_predictions, target_names=["healthy_loan","high_risk_loan"]))

                precision    recall  f1-score   support

  healthy_loan       1.00      1.00      1.00     18759
high_risk_loan       0.87      0.89      0.88       625

      accuracy                           0.99     19384
     macro avg       0.94      0.94      0.94     19384
  weighted avg       0.99      0.99      0.99     19384



### Step 4: Answer the following question.

**Question:** How well does the logistic regression model predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** The model performs well with a balanced accuracy score of 94.4 %.  However, the prediction of healthy loans precision is 100% compared to prediction of high-risk loans precision of 87%.  There are approximately 18,000 more low-risk loans than high-risk loans and this imbalance in instances per classifier suggests that high-risk loan prediction precision would improve as instances increase.  

---

## Predict a Logistic Regression Model with Resampled Training Data

### Step 1: Use the `RandomOverSampler` module from the imbalanced-learn library to resample the data. Be sure to confirm that the labels have an equal number of data points. 

In [33]:
# Import the RandomOverSampler module form imbalanced-learn
from imblearn.over_sampling import RandomOverSampler

# Instantiate the random oversampler model
# # Assign a random_state parameter of 1 to the model
ros_model = RandomOverSampler(random_state=1)

# Fit the original training data to the random_oversampler model
x_resampled, y_resampled = ros_model.fit_resample(x_train,y_train)

In [35]:
# Count the distinct values of the resampled labels data
y_resampled.value_counts()

0    56277
1    56277
Name: loan_status, dtype: int64

### Step 2: Use the `LogisticRegression` classifier and the resampled data to fit the model and make predictions.

In [36]:
# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
resample_model = LogisticRegression(solver='lbfgs', random_state=1)

# Fit the model using the resampled training data
resample_model.fit(x_resampled,y_resampled)

# Make a prediction using the testing data
resample_model_predictions = resample_model.predict(x_test)

### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [37]:
# Print the balanced_accuracy score of the model 
balanced_accuracy_score(y_test,resample_model_predictions)

0.9959744975744975

In [38]:
# Generate a confusion matrix for the model
confusion_matrix(y_test,resample_model_predictions)

array([[18668,    91],
       [    2,   623]])

In [39]:
# Print the classification report for the model
print(classification_report(y_test,resample_model_predictions))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     18759
           1       0.87      1.00      0.93       625

    accuracy                           1.00     19384
   macro avg       0.94      1.00      0.96     19384
weighted avg       1.00      1.00      1.00     19384



### Step 4: Answer the following question


**Question:** How well does the logistic regression model, fit with oversampled data, predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** The model performs well with a balanced accuracy score of 99.6 %. However, the prediction of healthy loans precision is 100% compared to prediction of high-risk loans precision of 87%. 
Notably, the logistic regression model with oversampled data has a recall recall rate than the previous model.  This could prevent high risk loans from slipping through the cracks and becoming costly unhealthy loans.  